### *IPCC SR15 scenario assessment*

<img style="float: right; height: 80px; padding-left: 20px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Assessment of underlying drivers and assumptions

This notebook contains the assessment of underlying drivers and assumptions of the scenario ensemble  
in **Section 2.3.1** and **Figure 2.4** for the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import io
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

from utils import boxplot_by_cat

## Import scenario data, categorization and specifications files

The metadata file with scenario categorisation and quantitative indicators can be downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).  
Alternatively, it can be re-created using the notebook `sr15_2.0_categories_indicators`.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r2.0.xlsx')

In [ ]:
sr1p5.load_meta('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream, Loader=yaml.FullLoader)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
cats_15 = specs.pop('cats_15')
marker= specs.pop('marker')

## Set specifications for figures and statistics

First, set the list of years included in the plots. Then, define an auxiliary dictionary and function for easier display.

In [ ]:
filter_args = dict(df=sr1p5, category=cats, marker=None, join_meta=True)

## Downselect scenario ensemble to years interest for this assessment

In [ ]:
full_horizon = range(2010, 2101, 10)

In [ ]:
df = sr1p5.filter(category=cats, year=full_horizon)

## Set plotting settings for illustrative pathways

In [ ]:
_rc = {
    'S1': dict(linewidth=3, linestyle='--'),
    'S2': dict(linewidth=4, linestyle=':'),
    'S5': dict(linewidth=3, linestyle='-.'),
    'LED': dict(linewidth=3, linestyle='-'),
}

In [ ]:
def line_plot_with_markers(ax, _df, name, panel):
    _data = _df.filter(category=cats_15, keep=False).data
    ax.plot(pyam.plotting.reshape_line_plot(_data, 'year', 'value'),
            color='lightgrey')
    ax.scatter(x=[], y=[], c='lightgrey', label='all scenarios')
    y_pos = max(_data.value) - 0.05 * (max(_data.value) - min(_data.value))

    _data = _df.filter(category=cats_15).data
    ax.plot(pyam.plotting.reshape_line_plot(_data, 'year', 'value'),
            color='xkcd:baby blue')
    ax.scatter(x=[], y=[], c='xkcd:baby blue', label='1.5°C pathways')

    for m in marker:
        _data = _df.filter(marker=m).data
        if not _data.empty:
            ax.plot(pyam.plotting.reshape_line_plot(_data, 'year', 'value'),
                    color='xkcd:darkish blue', **_rc[m], label=m)
    
    ax.set_ylabel('{} ({})'.format(name, _data.unit.unique()[0]))
    pyam.plotting.set_panel_label('({})'.format(panel), ax=ax)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8, 6))

pop = df.filter(variable='Population')
pop.convert_unit({'million': ['billion', 1/1000]}, inplace=True)
line_plot_with_markers(ax[0][0], pop, 'Population', 'a')

gdp = df.filter(variable='GDP|PPP')
gdp.convert_unit({'billion US$2010/yr': ['trillion US$2010/yr', 1/1000]}, inplace=True)
line_plot_with_markers(ax[0][1], gdp, 'Gross World Product', 'b')

final = df.filter(variable='Final Energy')
line_plot_with_markers(ax[1][0], final, 'Final Energy Demand', 'c')

food =  df.filter(variable='Food Demand')
line_plot_with_markers(ax[1][1], food, 'Food Demand', 'd')

ax[0][0].legend(loc=1)
fig.tight_layout()

In [ ]:
fig.savefig('output/fig2.4_drivers_assumptions.png')

## Export timeseries data to `xlsx`

In [ ]:
writer = pd.ExcelWriter('output/fig2.4_data_table.xlsx')
pyam.utils.write_sheet(writer, 'population', pyam.filter_by_meta(pop.timeseries(), **filter_args),
                       index=True)
pyam.utils.write_sheet(writer, 'gdp', pyam.filter_by_meta(gdp.timeseries(), **filter_args),
                       index=True)
pyam.utils.write_sheet(writer, 'final energy', pyam.filter_by_meta(final.timeseries(), **filter_args),
                       index=True)
pyam.utils.write_sheet(writer, 'food demand', pyam.filter_by_meta(food.timeseries(), **filter_args),
                       index=True)
writer.save()